In [82]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from configure import g_key

# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [83]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Busselton,32,AU,0,58,-33.65,115.33,55.00,6.91
1,Arraial do Cabo,20,BR,4,55,-22.97,-42.03,86.00,9.17
2,Atuona,48,PF,/,77,-9.80,-139.03,81.61,8.08
3,Cape Town,100,ZA,1,63,-33.93,18.42,66.99,6.93
4,Jamestown,90,US,5,64,42.10,-79.24,39.99,9.17
...,...,...,...,...,...,...,...,...,...
588,Bereda,100,ES,NaN,77,43.27,-7.54,62.01,5.82
589,Sioux Lookout,5,CA,NaN,41,50.10,-91.92,21.20,9.17
590,Tsaratanana,0,MG,NaN,49,-16.78,47.65,76.26,10.00
591,Broken Hill,66,AU,NaN,29,-31.95,141.43,78.80,10.29


In [84]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"].astype(float)

In [85]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [86]:
# vacay_df = cities_df.loc[cities_df["Max Temp"]<80, ["Max Temp"]>70, ["Wind Speed"]<10]
temp = (cities_df["Max Temp"]<80) & (cities_df["Max Temp"]>70)
wind = (cities_df["Wind Speed"]<10)
cloud = (cities_df["Cloudiness"]==0)

cities_df = cities_df[temp & wind & cloud]
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
77,Iquique,0,CL,NaN,60,-20.22,-70.14,71.60,6.93
215,Mananjary,0,MG,NaN,78,-21.22,48.33,78.21,7.36
311,Jardim,0,BR,NaN,38,-21.48,-56.14,70.56,7.40
469,San Isidro,0,PE,NaN,61,-12.12,-77.05,78.80,5.82
533,Reconquista,0,AR,NaN,34,-29.15,-59.65,70.47,4.70


In [87]:
cities_df["Hotel Name"] = ""
cities_df

/Users/mansi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
77,Iquique,0,CL,NaN,60,-20.22,-70.14,71.60,6.93,
215,Mananjary,0,MG,NaN,78,-21.22,48.33,78.21,7.36,
311,Jardim,0,BR,NaN,38,-21.48,-56.14,70.56,7.40,
469,San Isidro,0,PE,NaN,61,-12.12,-77.05,78.80,5.82,
533,Reconquista,0,AR,NaN,34,-29.15,-59.65,70.47,4.70,


In [88]:
hotel_df = pd.DataFrame(cities_df, columns = ["City", "Country", "Lat", "Lng", "Hotel Name"])
hotel_df

# cities_df

,City,Country,Lat,Lng,Hotel Name
77,Iquique,CL,-20.22,-70.14,
215,Mananjary,MG,-21.22,48.33,
311,Jardim,BR,-21.48,-56.14,
469,San Isidro,PE,-12.12,-77.05,
533,Reconquista,AR,-29.15,-59.65,


In [92]:
# cities_df.loc["Lat"]

In [125]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f" {lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
#     hotel_location = response.json
#     print(hotel_location)
    results = response['results']
    
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Closest hotel is Backpacker`s Hostel Iquique.
Closest hotel is Vahiny Lodge.
Closest hotel is Hotel Estância.
Closest hotel is Radisson Hotel Decapolis Miraflores.
Closest hotel is Grand Hotel.


In [126]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
77,Iquique,CL,-20.22,-70.14,Backpacker`s Hostel Iquique
215,Mananjary,MG,-21.22,48.33,Vahiny Lodge
311,Jardim,BR,-21.48,-56.14,Hotel Estância
469,San Isidro,PE,-12.12,-77.05,Radisson Hotel Decapolis Miraflores
533,Reconquista,AR,-29.15,-59.65,Grand Hotel


In [127]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [128]:
# fig.add_layer(info_box_template)
# fig
figs = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# fig.add_layer(hotel_layer)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))